In [48]:
import tensorflow as tf
import numpy as np

In [49]:
x = tf.Variable(np.array([3.]))
y = x ** 4 + x**2

In [50]:
optimizer = tf.train.GradientDescentOptimizer(0.05)
train = optimizer.minimize(y)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()
# Launch the graph.
with tf.Session() as sess:
    sess.run(init)
    # Fit the line.
    for step in range(201):
        if step % 20 == 0:
            print(step, sess.run(x), sess.run(y))
        sess.run(train)

0 [ 3.] [ 90.]
20 [ 0.07725829] [ 0.00600447]
40 [ 0.00932879] [  8.70338876e-05]
60 [ 0.00113405] [  1.28606939e-06]
80 [ 0.00013787] [  1.90091594e-08]
100 [  1.67622231e-05] [  2.80972124e-10]
120 [  2.03789494e-06] [  4.15301579e-12]
140 [  2.47760441e-07] [  6.13852362e-14]
160 [  3.01218846e-08] [  9.07327931e-16]
180 [  3.66211784e-09] [  1.34111070e-17]
200 [  4.45228021e-10] [  1.98227990e-19]


In [55]:
import autograd
from autograd import numpy as anp

In [56]:
x = anp.array([3], dtype='f8')
def y(x):
    return x **4 + x**2

In [57]:
type(autograd.grad(y))

function

In [53]:
from scipy.optimize import minimize

In [54]:
r = minimize(y, x, jac=autograd.grad(y), hess=autograd.hessian(y))
print(r)

      fun: 1.902044886389921e-18
 hess_inv: array([[ 0.49997778]])
      jac: array([ -2.75829287e-09])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 9
     njev: 12
   status: 0
  success: True
        x: array([ -1.37914643e-09])


/project/projectdirs/m779/yfeng1/envs/tensorflow/lib/python3.5/site-packages/scipy/optimize/_minimize.py:385: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)
